In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import time

In [ ]:
# Libs
import torch
from rembg import remove, new_session
from ultralytics import YOLO
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

from process.get_features import compute_depth, compute_pose_and_mask, save_keypoints, joints, compute_mask
from process.process_video import inpaint_video, process_video_frames

In [ ]:
RESULTS_FOLDER = "results/"
INPAINTING_FOLDER = RESULTS_FOLDER + "inpainting/"
KEYPOINTS_FOLDER = RESULTS_FOLDER + "keypoints/"
FRAMES_FOLDER = RESULTS_FOLDER + "frames_results/"

if not os.path.exists(RESULTS_FOLDER):
    os.makedirs(RESULTS_FOLDER)
if not os.path.exists(INPAINTING_FOLDER):
    os.makedirs(INPAINTING_FOLDER)
if not os.path.exists(KEYPOINTS_FOLDER):
    os.makedirs(KEYPOINTS_FOLDER)
if not os.path.exists(FRAMES_FOLDER):
    os.makedirs(FRAMES_FOLDER)

In [ ]:
VIDEO_FOLDER = "data/videos/"
videos = os.listdir(VIDEO_FOLDER)
print(len(videos))

# Inpainting

In [ ]:
for video in videos:
    print("Processing video: ", video)
    output_video_path = INPAINTING_FOLDER + video.replace(".mp4", "_inpainted.mp4")
    if os.path.exists(output_video_path):
        print("Video already processed")
        continue
    keypoints = inpaint_video(video_path = VIDEO_FOLDER + video, 
                              output_fp = output_video_path,
                              MAX_COUNT = None)

    output_keypoints_path = KEYPOINTS_FOLDER + video.replace(".mp4", "_keypoints.csv")
    save_keypoints(keypoints, output_keypoints_path)

# Get mask and depth frames

In [ ]:
for video in videos:
    print("Processing video: ", video)
    process_video_frames(VIDEO_FOLDER + video, FRAMES_FOLDER, show_output = False)

# Create particles video

In [ ]:
FRAMES_FOLDER = "data/particle_frames/"
frames_folders = os.listdir(FRAMES_FOLDER)

In [ ]:
for filename in frames_folders:
    output = RESULTS_FOLDER + "particles_videos/" + filename + ".mp4"
    if os.path.exists(output):
        print(f"Video {filename} already processed")
        continue
    os.chdir(FRAMES_FOLDER + filename)
    command = 'ffmpeg -i LS_Niagara.%04d.jpeg -c:v libx264 -vf "scale=1920:1080" ' + output
    print(command)
    print(os.system(command))